# Load Data and Preprocessing

In [ ]:
import pandas as pd
import os
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt

# Set the default DPI
plt.rcParams['figure.dpi'] = 100  

well_info = pd.read_csv('well-loc.tsv', sep='\t')

# Path to the sensor data directory
sensor_data_path = 'sensor-data'

# List all TSV files in the directory
sensor_data_files = [f for f in os.listdir(sensor_data_path) if f.endswith('.tsv')]

# Sort the sensor_data_files list
sensor_data_files.sort(key= lambda x: int(x.split('.')[0]))

# Load and concatenate all sensor data files into one DataFrame
sensor_data_list = [pd.read_csv(os.path.join(sensor_data_path, file), sep='\t',
                                na_values="-9999") for file in sensor_data_files]

# Remove the data point with NaN value
well_info = well_info.dropna()
for sensor_data in sensor_data_list:
    sensor_data.dropna(inplace=True)

# Reset the index of the well_loc DataFrame to Well, X, Y
well_info.rename(columns={'井': 'Well'}, inplace=True)

# Reset the index of the sensor data DataFrame to Depth, Porosity, Hydrate Saturation
for idx, _ in enumerate(sensor_data_list):
	sensor_data_list[idx].columns = ['Depth', 'Porosity', 'Hydrate Saturation']

print(well_info.head())  # Display the first few rows to verify it's loaded correctly
print(sensor_data_list[0].head())  # Display the first few rows to verify it's loaded correctly

In [ ]:
# Calculate the percentage of the data points contains the negative value in sensor data
negative_data = []
for idx, sensor_data in enumerate(sensor_data_list):
    condition = (sensor_data['Porosity'] < 0) | (sensor_data['Hydrate Saturation'] < 0)
    negative_data.append(len(sensor_data[condition]) / len(sensor_data))

# Plot the number of negative data points
fig = go.Figure(data=[go.Bar(x=sensor_data_files, y=negative_data)])
fig.update_layout(title_text='Number of negative data points in each sensor data file')
fig.show()


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Normalize the sensor data
scaler = MinMaxScaler()
for idx, sensor_data in enumerate(sensor_data_list):
    sensor_data[['Porosity', 'Hydrate Saturation']] = scaler.fit_transform(sensor_data[['Porosity', 'Hydrate Saturation']])

# Estimate Resource Distribution

In [ ]:
def estimate_resource(sensor_data: pd.Series) -> float:
	"""Estimate the resource at a given location based on sensor data"""
	# Get the Porosity and the Hydrate saturation
	porosity = sensor_data['Porosity']
	hydrate_saturation = sensor_data['Hydrate Saturation']

	valid_volume = 1 # Assume the valid volume is 1 cubic meter
	factor = 155 # Assume the factor is 155

	# Calculate the resource estimate
	return valid_volume * porosity * hydrate_saturation * factor

In [ ]:
esti_data_list: list[pd.DataFrame] = []

# Calculate the resource estimate for each sensor data in each depth
for sensor_data in sensor_data_list:
    estimation = pd.DataFrame()
    estimation['Estimated Resources'] = sensor_data.apply(estimate_resource, axis=1)
    estimation['Depth'] = sensor_data['Depth']
    esti_data_list.append(estimation)

esti_data_list[0]

# Kriging 3D Interpolation

### Define Functions

In [ ]:
def compose_krige3D(name: str, sample_frac: float, step_sizes: tuple, variogram: str):
    from pykrige.ok3d import OrdinaryKriging3D

    # Make kriging interpolation for Porosity  individually
    data_frames = []
    for i, df in enumerate(sensor_data_list):
        df['X'] = well_info.loc[i, 'X']
        df['Y'] = well_info.loc[i, 'Y']
        df['Well'] = well_info.loc[i, 'Well']
        df['Total Resources'] = well_info.loc[i, 'Total Resources']
        data_frames.append(df)

    combined_data = pd.concat(data_frames, ignore_index=True)

    sampled_data = combined_data.sample(frac=sample_frac, random_state=1)  # random_state for reproducibility

    X = sampled_data['X'].values
    Y = sampled_data['Y'].values
    Z = sampled_data['Depth'].values
    values = sampled_data[name].values

    # Create the 3D Kriging model
    ok3d = OrdinaryKriging3D(
        X,
        Y,
        Z,
        values,
        variogram_model=variogram,  # You might need to experiment with different models
        verbose=True,
        enable_plotting=True,
    )
    
    # Define grid points for interpolation
    grid_x = np.arange(min(X)-200, max(X)+200, step_sizes[0])
    grid_y = np.arange(min(Y)-200, max(Y)+200, step_sizes[1])
    grid_z = np.arange(min(Z)-1, max(Z)+1, step_sizes[2])

    return ok3d, (grid_x, grid_y, grid_z)

In [ ]:
def plot_3D_mesh(t, grid_x, grid_y, grid_z, title: str):
    GZ, GY, GX = np.meshgrid(grid_z, grid_y, grid_x, indexing='ij')
    print(GX.shape, GY.shape, GZ.shape)  # 应该输出 (12, 108, 88) 或类似，取决于具体的 grid_x, grid_y, grid_z

    fig = go.Figure(
        data=[go.Volume(
            x=GX.flatten(),
            y=GY.flatten(),
            z=GZ.flatten(),
            value=t.flatten(),
            isomin=t.min(),
            isomax=t.max(),
            opacity=0.5,  # Adjust the opacity to your liking
            surface_count=15,  # Adjust the number of isosurfaces
        )])

    # Update the layout of the plot
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis=dict(
                title='Depth',
                autorange='reversed'  # Automatically reverse the z-axis
            )
        )
    )

    # Show the plot
    fig.show()

### Kriging Interpolation of Total Resources

In [ ]:
ok3d, grid = compose_krige3D("Total Resources", 0.8, (10, 10, 1), 'linear')

In [ ]:
t_res, ss3d_res = ok3d.execute('grid', *grid)

In [ ]:
plot_3D_mesh(t_res, *grid, "Kriging Interpolation of Total Resources")
plot_3D_mesh(ss3d_res, *grid, "Kriging Varience of Total Resources")

### Kriging Interpolation of Porosity

In [ ]:
ok3d, grid = compose_krige3D("Porosity", 0.8, (10, 10, 1), 'linear')

In [ ]:
t_poro, ss3d_poro = ok3d.execute('grid', *grid)

In [ ]:
plot_3D_mesh(t_res, *grid, "Kriging Interpolation of Porosity")
plot_3D_mesh(ss3d_poro, *grid, "Kriging Varience of Porosity")